In [1]:
import pandas as pd

In [5]:
id_map = pd.read_csv('../data/id_map.csv', usecols = ['id', 'komuna'], encoding = "ISO-8859-1")

In [18]:
df = pd.read_csv('../data/komunat_new_cases.csv')
df['date'] = pd.to_datetime(df['date'])

In [31]:
df_all = pd.DataFrame()

for index, row in id_map.iterrows():
    komuna = row['komuna']
    if 'Enclave' not in komuna:
        tmp = df.loc[df['komuna'].str.contains(komuna.lower())]
        tmp['komuna'] = komuna
        # set date as new index
        tmp = tmp.set_index('date')
        # create new date index from 2020-07-08 to 2020-06-18
        new_index = pd.date_range(start='2020-06-18', end='2020-07-08')
        # reindex
        new_df = tmp.reindex(new_index, fill_value = 0)
        new_df['komuna'] = komuna
        new_df['id'] = row['id']
        df_all = df_all.append(new_df.reset_index(), ignore_index = True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [34]:
df_all = df_all.rename(columns={"index": "date"})
df_all

,date,komuna,new,id
0,2020-06-18,Deçan,0,0
1,2020-06-19,Deçan,0,0
2,2020-06-20,Deçan,0,0
3,2020-06-21,Deçan,0,0
4,2020-06-22,Deçan,0,0
...,...,...,...,...
793,2020-07-04,Mitrovicë e Veriut,0,39
794,2020-07-05,Mitrovicë e Veriut,0,39
795,2020-07-06,Mitrovicë e Veriut,0,39
796,2020-07-07,Mitrovicë e Veriut,0,39


In [35]:
# add cords
df_mun = pd.read_csv('../data/kosovo-municipalities.csv')

In [36]:
df_all["komuna_raw"] = df_all["komuna"].values
# remove special characters
df_all["komuna_raw"].replace("ë", "e", inplace=True, regex=True)
df_all["komuna_raw"].replace("ç", "c", inplace=True, regex=True)

# Merge with coordinates after you have everything
df = pd.merge(df_all, df_mun[["X", "Y", "nam"]], how="left", left_on = "komuna_raw", right_on = "nam")

In [38]:
df.drop("nam",inplace=True,axis=1)

In [39]:
df.to_csv('../data/covid-komunat.csv')